## Initial setup and imports

In [1]:
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

## Initial Data Exploration

In [9]:
element_tree = ET.parse(os.path.join('data', 'ds.xml'))
xml_root = element_tree.getroot()
data_list = []
for establishment in xml_root:
    establistment_dist = {}
    has_inspection_data = False
    for tag in establishment:
        if tag.tag == 'INSPECTION':
            has_inspection_data = True
        else:
            establistment_dist[tag.tag] = tag.text
    if has_inspection_data:
        inspection_dict = {}
        inspections = establishment.findall('INSPECTION')
        for inspection in inspections:
            for tag in inspection:
                if tag.tag != 'INFRACTION':
                    inspection_dict[tag.tag] = tag.text
            data_list.append({**establistment_dist, **inspection_dict})
    else:
        data_list.append(establistment_dist)
dinesafe_df = pd.DataFrame(data_list)

dinesafe_df

,ID,NAME,TYPE,ADDRESS,LATITUDE,LONGITUDE,STATUS,DATE
0,9008018,'K' STORE,Food Store (Convenience/Variety),99 CARLTON ST,43.66205,-79.37747,Pass,2019-03-29
1,10510325,0109 Dessert + Chocolate,Restaurant,"2190 MCNICOLL AVE, -109",43.81477,-79.29491,Pass,2019-10-25
2,10500438,1 PLUS 1 PIZZA,Food Take Out,361 OAKWOOD AVE,43.68725,-79.43842,Conditional Pass,2019-03-04
3,10500438,1 PLUS 1 PIZZA,Food Take Out,361 OAKWOOD AVE,43.68725,-79.43842,Pass,2019-03-08
4,10500438,1 PLUS 1 PIZZA,Food Take Out,361 OAKWOOD AVE,43.68725,-79.43842,Pass,2019-10-24
...,...,...,...,...,...,...,...,...
40923,10691709,fimi Kitchens,Restaurant,2958 ISLINGTON AVE,43.75729,-79.57019,Pass,2020-02-03
40924,10429136,iQ FOOD CO.,Restaurant,181 BAY ST,43.64748,-79.37849,Pass,2019-06-21
40925,10429136,iQ FOOD CO.,Restaurant,181 BAY ST,43.64748,-79.37849,Pass,2019-11-15
40926,10696752,iQx,Restaurant,55 AVENUE RD,43.67121,-79.39441,Pass,2020-01-02
